# Adding New Model Classes to MakeTables

Adding support for a new model class is super simple! You simply have to implement the `ModelExtractor` protocol defined in the `extractors.py` script and register it.

## Dev Environment Setup

To get started, we encourage you to set up a development environment, which starts by installing the package manager of our choice, `pixi`: 

1. Install [pixi](https://pixi.sh) by following the steps describe on their [installation page](https://pixi.sh/latest/installation/#__tabbed_1_1).

2. Clone maketables and create a dev environment with your package:
```bash
git clone git@github.com:py-econometrics/maketables.git # SSH
git clone https://github.com/py-econometrics/maketables.git # https
cd maketables
# create a new dev env and give it a name_of_new_env and add the
# packge for which you want to add a method name_of_model_package
pixi add --pypi --feature name_of_new_env name_of_model_package
# activate the new env: 
pixi shell -e name_of_new_env
```
Now you are good to go!

## Example: Statsmodels OLS Extractor

Below we attach a simplified version of the model extractor protocol for statsmodels, which provides a good blueprint for the addition of other models. 
After you have implemented it, please don't forget to update the `SupportedModelClasses.md` and `readme.md`!

```python
from maketables.extractors import register_extractor, _get_attr
import pandas as pd

# Check if statsmodels is installed
try:
    from statsmodels.regression.linear_model import RegressionResultsWrapper
    HAS_STATSMODELS = True
except ImportError:
    HAS_STATSMODELS = False
    RegressionResultsWrapper = ()  # empty tuple for isinstance check


class MyStatsmodelsExtractor:
    """Extractor for statsmodels OLS results."""
    
    # dict that translates between maketables model names (keys) 
    # and statsmodels attributes
    STAT_MAP = {
        "N": "nobs",
        "r2": "rsquared",
        "adj_r2": "rsquared_adj",
        "aic": "aic",
        "bic": "bic",
        "fvalue": "fvalue",
        "se_type": "cov_type",
    }
    
    def can_handle(self, model) -> bool:
        # check if statsmodels is installed
        if not HAS_STATSMODELS:
            return False
        return isinstance(model, RegressionResultsWrapper)
    
    def coef_table(self, model) -> pd.DataFrame:
        # Return coefficient table with canonical column names: b, se, p.
        # These tokens can be referenced directly in ETable's coef_fmt string.
        # Any additional columns (e.g., confidence intervals) can also be included by just adding a 
        # column to the df named with the respective token that the user can specify in thr format string.
        df = pd.DataFrame({
            "b": model.params,
            "se": model.bse,
            "t": model.tvalues,
            "p": model.pvalues,
        })
        
        df.index.name = "Coefficient"
        return df
    
    def depvar(self, model) -> str:
        # set the name of the dependent variable
        return getattr(model.model, "endog_names", "y")
    
    def fixef_string(self, model) -> str | None:
        # set the values of fixed effects as a string 
        # separated by a '+', ie 'f1+f2'. Only when 
        # fixed effects are supported
        return None

    def vcov_info(self, model) -> dict:
        # retrieve information on how the vcov matrix is computed
        return {"vcov_type": getattr(model, "cov_type", None), "clustervar": None}

    def var_labels(self, model) -> dict | None:
        # Extract variable labels from the model's data DataFrame when available.
        # Can be set to None for a MVP implementation
        return None

    # the remaining two methods can just be copied as stated below: 

    def stat(self, model: Any, key: str) -> Any:
        'Extract a statistic using STAT_MAP.'
        spec = self.STAT_MAP.get(key)
        if spec is None:
            return None
        val = _get_attr(model, spec)
        if key == "N" and val is not None:
            try:
                return int(val)
            except Exception:
                return val
        return val

    def supported_stats(self, model: Any) -> set[str]:
        'Return set of statistics available.'
        return {
            k for k, spec in self.STAT_MAP.items() if _get_attr(model, spec) is not None
        }


# Register at the bottom of the script: 
if HAS_STATSMODELS:
    register_extractor(MyStatsmodelsExtractor())
```

## Methods Summary

### Required Methods

| Method | Returns | Purpose |
|--------|---------|--------|
| `can_handle(model)` | `bool` | Return `True` if this extractor handles the model type |
| `coef_table(model)` | `DataFrame` | Columns (canonical tokens): `b` (estimate), `se` (std. error), `p` (p-value), optionally `t` (t-statistic). May include additional columns like `ci95l`, `ci95u`, etc. |
| `depvar(model)` | `str` | Dependent variable name |
| `fixef_string(model)` | `str \| None` | Fixed effects spec (e.g., `"entity+time"`), `None` if no FE support |
| `vcov_info(model)` | `dict` | Keys: `vcov_type`, `clustervar` |
| `stat(model, key)` | `Any` | Extract stat by key: `N`, `r2`, `adj_r2`, `se_type`, etc. |
| `var_labels(model)` | `dict \| None` | Variable name → label mapping |
| `supported_stats(model)` | `set[str]` | Set of available stat keys |

## Alternative: Plug-in Extractor Format

If you maintain your own package and want to make it compatible with `maketables` without requiring an extractor implementation in maketables itself, you can use the **plug-in extractor format**. 

Simply add specific attributes and methods to your model result class, and maketables will automatically detect and use them. This approach requires zero coupling between your package and maketables.

### Quick Start: Implementing the Plug-in Format

Add these attributes/methods to your model result class:

```python
import pandas as pd

class MyModelResult:
    """Your package's regression result class."""
    
    @property
    def __maketables_coef_table__(self) -> pd.DataFrame:
        """
        Required: Return coefficient table with canonical column names.
        
        Columns needed: 'b', 'se', 'p'
        Columns optional: 't', 'ci95l', 'ci95u', etc.
        """
        return pd.DataFrame({
            'b': self.params,
            'se': self.bse,
            't': self.tvalues,
            'p': self.pvalues,
        })
    
    def __maketables_stat__(self, key: str) -> float | str | int | None:
        """
        Required: Return model statistics by key.
        
        Common keys: 'N', 'r2', 'adj_r2', 'aic', 'bic', 'rmse', 'fvalue', 'se_type'
        """
        stats = {
            'N': self.nobs,
            'r2': self.rsquared,
            'adj_r2': self.rsquared_adj,
        }
        return stats.get(key)
    
    @property
    def __maketables_depvar__(self) -> str:
        """Return the dependent variable name."""
        return self.model.endog_names
    
    @property
    def __maketables_fixef_string__(self) -> str | None:
        """Optional: Return fixed effects as '+'-separated string, or None."""
        return None  # or e.g., "firm+year" for models with fixed effects
    
    @property
    def __maketables_var_labels__(self) -> dict[str, str] | None:
        """Optional: Return variable name → label mapping, or None."""
        if hasattr(self.model, 'data') and hasattr(self.model.data, 'attrs'):
            return self.model.data.attrs.get('variable_labels')
        return None
    
    @property
    def __maketables_vcov_info__(self) -> dict[str, str] | None:
        """Optional: Return variance-covariance info, or None."""
        return {'se_type': getattr(self, 'cov_type', None)}
```

### Using Your Plug-in Compatible Model

Once your model class implements these attributes, users can use it directly with maketables:

```python
from mypackage import MyRegression
from maketables import ETable

# Fit your model
result = MyRegression(y, X)

# maketables automatically detects the plug-in format!
table = ETable(result)
table.save('my_table.tex')
```

### Advantages of the Plug-in Format

- **Zero dependency**: Your package doesn't need to import maketables
- **Zero maintenance**: No need to update when maketables changes
- **Self-contained**: All integration logic lives in your package
- **Progressive**: Implement only what you need; optional attributes have sensible defaults
- **Standard**: Use the `__maketables_*__` naming convention for clarity

For complete specifications, see `PLUGIN_EXTRACTOR_FORMAT.md`.